## 0. Loading dependences

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

## 1. Reading videos in folders to create output

In [2]:
CADENCE = 20            # cadence of new video (designed for 30fps)
BLUR_CADENCE = 120      # number of frames over which to blur
XSCALE = 1280           # horizontal scale of the video frame; must be accurate!!!
YSCALE = 720            # vertical scale of the video frame; must be accurate!!!
FPS = 30                # FPS of incoming and outgoing video
VLENGTH = 300           # length of one video in seconds
video_folders = ['path_to_folder1', \
                 'path_to_folder2'] \
                  # folders (in order) where to capture and glue video from

# recording the names of video files and understanding which ones to blur
video_files = []
blur_start = []
blur_end = []
for video_folder in video_folders:
    blur = 1
    for video_number in range (0,100,1):
        if (video_number < 10):
            video_name = os.path.join(video_folder,'LOCK000'+str(video_number)+'.mov')    # use any other related prefix/postfix
        else:
            video_name = os.path.join(video_folder,'LOCK00'+str(video_number)+'.mov')     # use any other related prefix/postfix
        if (os.path.isfile(video_name)):
            video_files.append(video_name)
            blur_start.append(blur)
            blur = 0
        else:
            blur = 1
blur_end = blur_start[1:]
blur_end.append(1)

# creating the output video
fourcc = cv2.cv.CV_FOURCC(*'XVID')
outvideo = cv2.VideoWriter('outputvideo.avi',fourcc, FPS, (XSCALE,YSCALE))

nvideos = len(video_files)
# iterating through the recognized files
for nvideo in range (0,nvideos,1):
    video_file = video_files[nvideo]
    # first, reading the incoming video to temporary array
    arraydim = int(1.05*float(VLENGTH)*FPS/CADENCE)
    videoarray = np.zeros([YSCALE,XSCALE,3,arraydim], dtype='uint8')
    nframes = 0
    nframes_orig = 0
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        ret, frame = cap.read()
        nframes_orig += 1
        if (ret == False): break
        if (nframes_orig % CADENCE) == 0:
            videoarray[:,:,:,nframes] = frame
            nframes += 1
    cap.release()
    videoarray = np.copy(videoarray[:,:,:,0:nframes])
    # second, blur the array if the flags indicate so
    if (blur_start[nvideo] == 1):
        for i in range (0,BLUR_CADENCE,1):
            videoarray[:,:,:,i] = (np.copy(videoarray[:,:,:,i])*float(i)/float(BLUR_CADENCE-1)).astype('uint8')
    if (blur_end[nvideo] == 1):
        for i in range (-1,-1*BLUR_CADENCE-1,-1):
            videoarray[:,:,:,i] = (np.copy(videoarray[:,:,:,i])*float((-1)*i-1)/float(BLUR_CADENCE-1)).astype('uint8')
    # third, record the array in video file
    for i in range (0,nframes,1):
        outvideo.write(videoarray[:,:,:,i])
    # printing the status
    print "PROCESSED: "+video_file

outvideo.release()